If you're opening this Notebook on colab, you will probably need to install the most recent versions of 🤗 Transformers and 🤗 Datasets. We will also need `scipy` and `scikit-learn` for some of the metrics. Uncomment the following cell and run it.

In [2]:
! pip install transformers
! pip install datasets
! pip install scipy sklearn
! pip install huggingface_hub

  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
     -------------------------------------- 268.0/268.0 kB 5.5 MB/s eta 0:00:00
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Using cached filelock-3.8.0-py3-none-any.whl (10 kB)
     ---------------------------------------- 3.3/3.3 MB 8.1 MB/s eta 0:00:00
  Using cached datasets-2.6.1-py3-none-any.whl (441 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     -------------------------------------- 115.1/115.1 kB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 19.5/19.5 MB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 10.0/10.0 MB 8.0 MB/s eta 0:00:00
  Using cached fsspec-2022.8.2-py3-none-any.whl (140 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached pytz-2022.5-py2.py3-none-any.whl (500 kB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): starte

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


If you're opening this notebook locally, make sure your environment has an install from the latest version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow:

First you have to create an access token on the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input your token.

In [5]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS and setup Git if you haven't already. Uncomment the following instructions and adapt with your name and email:

In [6]:
!apt install git-lfs
!git config --global user.email "idfassahata@gmail.com"
!git config --global user.name "fassahat"

'apt' is not recognized as an internal or external command,
operable program or batch file.


Make sure your version of Transformers is at least 4.16.0 since the functionality was introduced in that version:

In [7]:
import transformers

print(transformers.__version__)

4.23.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/text-classification).

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

![Widget inference on a text classification task](https://github.com/huggingface/notebooks/blob/main/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use Keras to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (a task with the same training set as `mnli` but different validation and test sets):

In [8]:
GLUE_TASKS = [
    "cola",
    "mnli",
    "mnli-mm",
    "mrpc",
    "qnli",
    "qqp",
    "rte",
    "sst2",
    "stsb",
    "wnli",
]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set these three parameters, then the rest of the notebook should run smoothly:

In [9]:
task = "sst2"
model_checkpoint = "distilbert-base-uncased"
model_cp = "bert-for-patents"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and the [🤗 Evaluate](https://github.com/huggingface/datasets) library to get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the `load_dataset` function from `datasets` and and the `load` function from `evaluate`. 

In [10]:
from datasets import load_dataset
! pip install evaluate
from evaluate import load

     ---------------------------------------- 72.9/72.9 kB 2.0 MB/s eta 0:00:00


With the exception of `mnli-mm`, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [9]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [34]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load("glue", actual_task)

from datasets import load_dataset

# imdb = load_dataset("imdb")
sentence_data = load_dataset("csv", data_files="C:/Study/Master Thesis/expaai_model/tmp/sentence_based_data_150K.csv")

Found cached dataset glue (C:/Users/ranax/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 744.99it/s]
Using custom data configuration default-5e158816d501c9db


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1003.90it/s]


Dataset csv downloaded and prepared to C:/Users/ranax/.cache/huggingface/datasets/csv/default-5e158816d501c9db/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 71.46it/s]


In [35]:
sentence_data_randomized = sentence_data["train"].shuffle(seed=42)
sentence_data_randomized_rows = sentence_data.num_rows['train']
train_end = round(sentence_data_randomized_rows * 0.80) #take atleast 80% samples for training

sentence_train_dataset = sentence_data_randomized.select([i for i in list(range(train_end))])
sentence_test_dataset = sentence_data_randomized.select([i for i in list(range(train_end, sentence_data_randomized_rows))])

In [36]:
# small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
# small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(3000))])
# print(small_train_dataset)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [37]:
# dataset

To access an actual element, you need to select a split first, then give an index:

In [38]:
# dataset["train"][0]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [39]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [40]:
show_random_elements(sentence_train_dataset)

,sentence,label
0,It is preferred that the surface-smooth layer be substantially free from a fine particle.,0
1,"Therefore, a therapeutic effect of PNA has not been confirmed in an experiment system using cells.",2
2,This drawback is because of a handle structure of bicycle as described thereafter.,2
3,"', 'When DCI format 3/3A including the TPC command for an A-SRS is transmitted and received only through a subframe or a PRB with specific time and time interval, the transmitting opportunity of a TPC command of DCI format 3/3A decreases.",1
4,"In addition, because the propellers are not deployed or unfolded in the folded mode, a volume of the drone may be minimized, and thus it may be convenient to store and carry the drone around.",1
5,"', 'In the above case, the water supply source may be a cooling-medium tank which stores the cooling medium, and the cooling-medium tank may be arranged in an upper part of the tower and open to a space inside the tower.",0
6,"', 'In the air conditioning indoor unit pertaining to the third aspect of the present invention, even when the outlet air whose air direction has been adjusted by the first air direction adjustment plate is horizontal or a little downward, it becomes upward air because of the Coanda effect, on it is not necessary for the air just after passage through the air outlet to be forcibly directed upward.",1
7,"', 'In the abovementioned intermediate structure unit for a secondary cell, the plurality of test structure units may include a test structure unit in which the charging layer is not formed between the first electrode layer and the second electrode layer.",0
8,"According to the embodiments of the invention, the vector map display system and method based on a map scale change can change the viewpoint of the map according to two or more viewpoints set therein.",1
9,"', 'Therefore, an object of the present invention is to provide a power supply apparatus for sliding structure for constantly supplying power from a fixed structure to a sliding structure, which can reduce the size, weight and cost of a sliding structure-side and/or a fixed structure-side power supplying structure, which can reliably prevent, with a simple structure, the interference between a wiring harness and the fixed structure when the sliding structure is fully-opened or half-opened, which can prevent the decrease in the bending durability of the wiring harness due to the prevention of said interference, and which can achieve smooth bending of the wiring harness during the opening operation and/or the closing operation of the sliding structure to improve the bending durability of the wiring harness.",2


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [41]:
metric

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=ref

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [42]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.375}

Note that `load` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

You can directly call this tokenizer on one sentence or a pair of sentences:

In [43]:
tokenizer("Hello, this is a sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2003, 1037, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [44]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [45]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {sentence_train_dataset[0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Accordingly, it is possible to prevent a disturbance to the intaglio printing plate, and the inks can be transferred evenly from the blanket to the intaglio printing plate.


We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the arguments `truncation=True` and `padding='longest`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model, and all inputs will be padded to the maximum input length to give us a single input array. A more performant method that reduces the number of padding tokens is to write a generator or `tf.data.Dataset` to only pad each *batch* to the maximum length in that batch, but most GLUE tasks are relatively quick on modern GPUs either way.

In [46]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [47]:
preprocess_function(sentence_train_dataset[:5])

{'input_ids': [[101, 11914, 1010, 2009, 2003, 2825, 2000, 4652, 1037, 16915, 2000, 1996, 20014, 8490, 12798, 8021, 5127, 1010, 1998, 1996, 10710, 2015, 2064, 2022, 4015, 18030, 2013, 1996, 8768, 2000, 1996, 20014, 8490, 12798, 8021, 5127, 1012, 102], [101, 1999, 2060, 2616, 1010, 1996, 5527, 4942, 29390, 2108, 1996, 4539, 1997, 2968, 2024, 3445, 1012, 102], [101, 1005, 1010, 1005, 1999, 2028, 6871, 7861, 5092, 21341, 1010, 2006, 1996, 3302, 1997, 1996, 20681, 6741, 1010, 1996, 20681, 2555, 1997, 1996, 2034, 6204, 7730, 2828, 2582, 2950, 2012, 2560, 2028, 2117, 1011, 2856, 6167, 4664, 2008, 3216, 2247, 1037, 2117, 8123, 2008, 2003, 2025, 5903, 2000, 1996, 2034, 8123, 1012, 102], [101, 1999, 2804, 1010, 2060, 3471, 5793, 2000, 2216, 2383, 6623, 3716, 1999, 1996, 2396, 2089, 2022, 13332, 1999, 3193, 1997, 4751, 4541, 1999, 1996, 2556, 11028, 1012, 102], [101, 1005, 1010, 1005, 2004, 2649, 2682, 1010, 2043, 1996, 4083, 5080, 2429, 2000, 1996, 2556, 7861, 5092, 21341, 16463, 2008, 1996, 219

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [48]:
pre_tokenizer_columns = set(sentence_train_dataset.features)
encoded_dataset_train = sentence_train_dataset.map(preprocess_function, batched=True)
encoded_dataset_test = sentence_test_dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset_train.features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

 97%|█████████▋| 29/30 [00:03<00:00,  8.95ba/s]

Columns added by tokenizer: ['attention_mask', 'input_ids']


In [49]:
encoded_dataset_train.features["label"]

Value(dtype='int64', id=None)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `TFAutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels).

In [50]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

num_labels = 3 if task.startswith("mnli") else 1 if task == "stsb" else 2
if task == "stsb":
    num_labels = 1
elif task.startswith("mnli"):
    num_labels = 3
else:
    num_labels = 2
    
# This next little bit is optional, but will give us cleaner label outputs later
# If you're using a task other than CoLA, you will probably need to change these
# to match the label names for your task!
id2label = {0: "0", 1: "1", 2: "2"}
label2id = {val: key for key, val in id2label.items()}

# model = TFAutoModelForSequenceClassification.from_pretrained(
#     model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id
# )

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_39', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

Next, we convert our datasets to `tf.data.Dataset`, which Keras understands natively. There are two ways to do this - we can use the slightly more low-level [`Dataset.to_tf_dataset()`](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.to_tf_dataset) method, or we can use [`Model.prepare_tf_dataset()`](https://huggingface.co/docs/transformers/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset). The main difference between these two is that the `Model` method can inspect the model to determine which column names it can use as input, which means you don't need to specify them yourself. Unless our samples are all the same length, we will also need to pass a `tokenizer` or `collate_fn` so that the `tf.data.Dataset` knows how to pad and combine samples into a batch.

In [51]:
validation_key = (
    "validation_mismatched"
    if task == "mnli-mm"
    else "validation_matched"
    if task == "mnli"
    else "validation"
)

tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset_train,
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset_test,
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

Next, we need to set up our optimizer and `compile()` our model. The `create_optimizer` function in the Transformers library creates a very useful `AdamW` optimizer with weight and learning rate decay. This performs very well for training most transformer networks - we recommend using it as your default unless you have a good reason not to! Note, however, that because it decays the learning rate over the course of training, it needs to know how many batches it will see during training.

Note that all models in `transformers` can pick a sensible loss function by default. To use this loss, simply do not pass a `loss` argument to `compile()`. Although the losses for GLUE tasks are usually just simple cross-entropy, this can be very helpful in models when the loss is intricate and contains multiple terms.

In some of our other examples, we use `jit_compile` to compile the model with [XLA](https://www.tensorflow.org/xla). In this case, we should be careful about that - because our inputs have variable sequence lengths, we may end up having to do a new XLA compilation for each possible length, because XLA compilation expects a static input shape! For small datasets, this will probably result in spending more time on XLA compilation than actually training, which isn't very helpful.

If you really want to use XLA without these problems (for example, if you're training on TPU), you can create a tokenizer with `padding="max_length"`. This will pad all of your samples to the same length, ensuring that a single XLA compilation will suffice for your entire dataset. Note that depending on the nature of your dataset, this may result in a lot of wasted computation on padding tokens!

In [52]:
from transformers import create_optimizer

num_epochs = 3
batches_per_epoch = len(encoded_dataset_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


The last thing to define is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier. The only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B).

In addition, let's wrap this metric computation function in a `KerasMetricCallback`. This callback will compute the metric on the validation set each epoch, including printing it and logging it for other callbacks like `TensorBoard` and `EarlyStopping`.

Why do it this way, though, and not just use a straightforward Keras Metric object? This is a good question - on this task, several of the metrics such as Accuracy are very straightforward, and it would probably make more sense to just use a Keras metric for those instead. However, we want to demonstrate the use of `KerasMetricCallback` here, because it can handle any arbitrary Python function for the metric computation. This turns out to be very important for other NLP tasks like summarization and translation, where standard metrics like `BLEU` and `ROUGE` are much more complex to compute, and often involve decoding tokens generated by the model to strings and comparing their similarity to target sentences. If you want to stop training once `ROUGE` scores on the validation set start to decline, then `KerasMetricCallback` is essential.

That said, if you're only interested in tasks like text classification with straightforward metrics, then by all means remove the `KerasMetricCallback` and use a Keras `Accuracy` metric instead!

With that out of the way, how do we actually use `KerasMetricCallback`? It's straightfoward: We simply define a function that computes metrics given a tuple of numpy arrays of predictions and labels, then we pass that, along with the validation set to compute metrics on, to the callback:

In [53]:
from transformers.keras_callbacks import KerasMetricCallback


def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

We can now finetune our model by just calling the `fit` method. Be sure to pass the TF datasets, and not the original datasets! We can also add a callback to sync up our model with the Hub - this allows us to resume training from other machines and even test the model's inference quality midway through training! Make sure to change the `username` if you do. If you don't want to do this, simply remove the callbacks argument in the call to `fit()`.

In [57]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

push_to_hub_model_id = "distillbert-base-uncased-finetuned-150k-patent-sentences-my-pc"

tensorboard_callback = TensorBoard(log_dir="./my_pc_distillbert_base_uncased_sentence_classification_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./my_pc_distillbert_base_uncased_sentence_classification_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
    save_strategy="epoch"
)

callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=callbacks
)

c:\Study\Master Thesis\expaai_model\BertPatent\my_pc_distillbert_base_uncased_sentence_classification_model_save is already a clone of https://huggingface.co/fassahat/distillbert-base-uncased-finetuned-150k-patent-sentences-my-pc. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node 'tf_distil_bert_for_sequence_classification_1/distilbert/transformer/layer_._2/attention/MatMul' defined at (most recent call last):
    File "c:\Users\ranax\anaconda3\envs\tf\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "c:\Users\ranax\anaconda3\envs\tf\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "c:\Users\ranax\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\ranax\anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "c:\Users\ranax\anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\ranax\anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

If you used the callback above, you can now share this model with all your friends, family or favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("your-username/my-awesome-model")
```

## Inference

Training a model is fun, but once it's trained you usually want to use it to get predictions on new data. Let's take a look at how to do that. Firstly, we'll load our trained model from the hub - this lets us resume the code from here without needing to rerun all the training above every time.

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# You can, of course, use your own username and model name here once you've pushed your model using the code above!
model = TFAutoModelForSequenceClassification.from_pretrained("fassahat/distill-bert-finetuned-150k-patent-sentences")
tokenizer = AutoTokenizer.from_pretrained("fassahat/distill-bert-finetuned-150k-patent-sentences")

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some layers from the model checkpoint at fassahat/distill-bert-finetuned-150k-patent-sentences were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at fassahat/distill-bert-finetuned-150k-patent-sentences and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Now, let's make up some sentences and see if the model can classify them properly! The first sentence is valid English, but the second one makes a grammatical mistake.

In [ ]:
sentences = [
    "The judge told the jurors to think carefully.",
    "The judge told that the jurors to think carefully."
]

To feed them into our model, we'll need to tokenize them and then get our model's predictions:

In [ ]:
tokenized = tokenizer(sentences, return_tensors="np", padding="longest")

outputs = model(tokenized).logits

classifications = np.argmax(outputs, axis=1)
print(classifications)

[1 0]


What do those label values mean? Let's use the `id2label` property set on our model to make them a little more comprehensible:

In [ ]:
classifications = [model.config.id2label[output] for output in classifications]
print(classifications)

['Valid', 'Invalid']


Looks right to me!

## Pipeline API

An alternative way to quickly perform inference with any model on the hub is to use the [Pipeline API](https://huggingface.co/docs/transformers/main_classes/pipelines), which abstracts away all the steps we did manually above. It will perform the preprocessing, forward pass and postprocessing all in a single object.

Let's showcase this for our trained model:

In [1]:
from transformers import pipeline

classifier = pipeline("text-classification", "fassahat/distilbert-base-uncased-finetuned-557k-patent-sentences", framework="tf")

Some layers from the model checkpoint at fassahat/distilbert-base-uncased-finetuned-557k-patent-sentences were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at fassahat/distilbert-base-uncased-finetuned-557k-patent-sentences and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

c:\Users\ranax\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ranax\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
sentences = [
    "The judge told the jurors to think carefully.",
    "The judge told that the jurors to think carefully."
]
classifier(sentences)

[{'label': '0', 'score': 0.8500396609306335},
 {'label': '0', 'score': 0.7877770066261292}]

And that's it - the code above is all you need to get classifications from your model in future! Note how the `id2label` property we set during training is automatically read by our pipeline to assign sensible names to the output classes.